# Creating a basic chat experience with kernel arguments

In this chat scenario, as the user talks back and forth with the bot, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. 

In [18]:
#r "nuget: Microsoft.SemanticKernel, 1.1.0"
#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.1.0

Let's define the prompt.

In [19]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

Register your semantic function

In [20]:
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Initialize your arguments

In [21]:
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

Chat with the Bot

In [25]:
var userInput = "Hi, I'm looking for cocktail suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

Update the history with the output and set this as the new input value for the next request

In [26]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);


User: Hi, I'm looking for cocktail suggestions
AI: Sure! What type of cocktail are you in the mood for?

User: Hi, I'm looking for cocktail suggestions
AI: Sure! What type of cocktail are you in the mood for?

User: Hi, I'm looking for cocktail suggestions
AI: Sure! What type of cocktail are you in the mood for?



Keep Chatting!

In [27]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;
    
    // Show the response
    Console.WriteLine(result);
};

In [30]:
await Chat("I would like a cocktail suggestion for breakfast. Please suggest one of them.");


User: I would like a cocktail suggestion for breakfast. Please suggest one of them.
AI: How about a classic mimosa? It's a refreshing combination of orange juice and champagne. Perfect for a brunch or breakfast cocktail.



In [29]:
await Chat("that sounds interesting, how do you make one?");


User: that sounds interesting, how do you make one?
AI: To make a mimosa, you will need:
- 1 part orange juice
- 1 part champagne or sparkling wine

Here's how to make it:
1. Pour the orange juice into a champagne flute or glass.
2. Slowly pour the champagne or sparkling wine into the glass.
3. Gently stir the mixture with a spoon.
4. Garnish with a slice of orange, if desired.
5. Serve and enjoy!

Remember to drink responsibly and enjoy your mimosa!



In [31]:
await Chat("Write a C# code and print hello world?");


User: Write a C# code and print hello world?
AI: Sure! Here's a simple C# code that prints "Hello, World!" to the console:

```csharp
using System;

class Program
{
    static void Main()
    {
        Console.WriteLine("Hello, World!");
    }
}
```

You can copy and paste this code into a C# compiler or IDE, and it will display "Hello, World!" when you run it.



In [32]:
await Chat("could you list some more I could try about the topic(s) mentioned?");


User: could you list some more I could try about the topic(s) mentioned?
AI: Sure! Here are a few more cocktail suggestions for you to try:

1. Bloody Mary - A classic brunch cocktail made with vodka, tomato juice, and various spices and seasonings.
2. Bellini - A refreshing cocktail made with peach puree and sparkling wine.
3. Screwdriver - A simple cocktail made with vodka and orange juice.
4. Irish Coffee - A warm and comforting cocktail made with coffee, Irish whiskey, sugar, and topped with whipped cream.
5. Piña Colada - A tropical cocktail made with rum, pineapple juice, and coconut cream.

These are just a few options, but there are countless cocktail recipes out there to explore. Enjoy experimenting and finding your favorites!



After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [14]:
Console.WriteLine(history);


User: Hi, I'm looking for cocktail suggestions
AI: Hi! I'd be happy to help with that. Are you looking for something specific, like a cocktail based on a particular spirit, or are you open to any suggestions?

User: I would like a cocktail suggestion for breakfast. Please list one.
AI: For a breakfast cocktail, I suggest trying a classic Mimosa. It's refreshing, easy to make, and pairs wonderfully with many breakfast dishes. Here's how you can make it:

### Mimosa Recipe

**Ingredients:**
- 2 ounces of chilled orange juice (freshly squeezed is best)
- 2 ounces of chilled Champagne or any sparkling wine
- Optional: a splash of Grand Marnier or triple sec for an extra kick
- Orange slice or twist, for garnish

**Instructions:**
1. Fill a Champagne flute halfway with the chilled orange juice.
2. Top it off with the chilled Champagne or sparkling wine.
3. If you're using, add a splash of Grand Marnier or triple sec.
4. Gently stir to combine.
5. Garnish with an orange slice or twist.

Enj